# **Yelp Rating Predictor**

Objective: predict Yelp! rating for newly opened bakery based on the information of other restaurants' rating provided by Yelp!

Use Multiple Lineal Regression model to decide what factors contribute the most to a high rating



# Import data and EDA

In [ ]:
import pandas as pd

: 